In [1]:
from requests import get
from requests.auth import HTTPBasicAuth
from IPython.core.interactiveshell import InteractiveShell
from pandas import DataFrame, read_sql_query
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv

from core.settings import setting



InteractiveShell.ast_node_interactivity = "all"

In [2]:
oauth = HTTPBasicAuth(setting.email_comcare,setting.password_comcare)

In [3]:
pay = {'xmlns':setting.xmlns, 'limit':setting.limit}
forms = get(setting.form_base_url,auth=oauth,params=pay).json()
__objects = forms["objects"]
__meta = forms["meta"]
print(__meta)

while __meta["next"]:
    __forms = get(f"""{setting.form_base_url}{__meta['next']}""",auth=oauth).json()
    __objects += __forms["objects"]
    __meta = __forms["meta"]

len(__objects)

{'limit': 1000, 'next': '?xmlns=http%3A%2F%2Fopenrosa.org%2Fformdesigner%2FDD8FE78F-C33C-4D7D-A63B-AEDFF3A85279&limit=1000&offset=1000', 'offset': 0, 'previous': None, 'total_count': 7125}


7126

In [4]:
pay_cases = {'type':setting.types,'limit':setting.caselimit}
form_cases = get(setting.case_base_url,auth=oauth,params=pay_cases).json()
__objects_cases = form_cases["objects"]
__meta_cases = form_cases["meta"]
print(__meta_cases)

while __meta_cases["next"]:
    __form_cases = get(f"""{setting.case_base_url}{__meta_cases['next']}""",auth=oauth).json()
    __objects_cases += __form_cases["objects"]
    __meta_cases = __form_cases["meta"]

len(__objects_cases)


{'limit': 5000, 'next': '?type=muso_household_2022&limit=5000&offset=5000', 'offset': 0, 'previous': None, 'total_count': 12533}


12533

In [5]:
beneficiare_direct = DataFrame(list(
    map(
        lambda k: {
            "userID":k['metadata']['userID'],
            "username":k['metadata']['username'],
            "timeEnd": k['metadata']['timeEnd'],
            "case_id": k['form']['case']['@case_id'],
            "houseold_number_2022": k['form']['household_number'],
            "db_name": k['form']['nom'],
            "db_first_name": k['form']['prenom']
        } 
        ,__objects
    )
))

beneficiare_direct.drop_duplicates('case_id',inplace=True)
beneficiare_direct.reset_index(drop=True)
#beneficiare_direct.case_id.value_counts()

beneficiare_direct.case_id.count()

,userID,username,timeEnd,case_id,houseold_number_2022,db_name,db_first_name
0,a95f2e5d43cf492986a6c228d66c9f3c,nljoseph,2022-03-16T13:54:27.227000Z,7d80ed09-c9c1-4ec7-ba3f-967e8b69a05e,0,,Rosemane
1,066a95de555443719f17ffe5732f9657,emichel,2022-03-16T14:53:44.828000Z,241b91e0-3ac4-4eee-8632-ff392e586a17,4,Saint Juste,Angeline
2,066a95de555443719f17ffe5732f9657,emichel,2022-03-16T14:52:12.250000Z,8b0dfc4b-16d0-4893-b44d-1417a0f5fcd6,1,,Onick
3,ece16e9f4d2c468e961d2b56ff621a2e,myvelise,2022-03-16T13:52:16.545000Z,6968dd8a08034725815a56e767c53e02,0,Joseph,Alex
4,ece16e9f4d2c468e961d2b56ff621a2e,myvelise,2022-03-16T13:51:15.560000Z,9121343969df4a6dad95b4e197597423,3,François,Olga
...,...,...,...,...,...,...,...
6910,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:41:26.240000Z,88881743-96fe-4e4b-b9e8-5bd30490ad8f,0,Diaquot,Widlet
6911,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:34:11.744000Z,7f6fc269-d750-44e1-b4cf-7446a14960a8,0,Darisma,John Evens
6912,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:26:50.145000Z,c2792c70-6b09-4f69-b2d4-69cc4498ed10,0,Guerrelus,Sony
6913,02a25cddec634621822ccaa4657f8246,mcarthur,2022-02-16T20:02:48.005000Z,724681f8-f8bd-4fb4-a0b4-c20c1c0b82d0,0,Vincent,Lovena


6915

In [6]:
beneficiare_indirect = DataFrame(list(
    map(
        lambda k: {
            "userID": k['user_id'],
            "ib_case_id": k['case_id'],
            "age": k['properties']['age'],
            "sexe": k['properties']['sexe'],
            "arv": k['properties']['arv'],
            "test": k['properties']['test'],
            "ib_case_id": k['case_id'],
            "parent_case_id": k['indices']['parent']['case_id'],
            "relationship": k['indices']['parent']['relationship'],
        } 
        ,__objects_cases
    )
))

beneficiare_indirect
beneficiare_indirect.ib_case_id.count()



,userID,ib_case_id,age,sexe,arv,test,parent_case_id,relationship
0,436b6c0759934988b5cf76ced1c46bfd,ca6f342b-7499-4ea3-b9ee-39bec46638a6,2,2,non,non,a81e3aa1b7f3470b8c064788445145e3,child
1,02a25cddec634621822ccaa4657f8246,4a475221-a783-418b-9fcc-42c7ca03a7bd,4,2,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
2,02a25cddec634621822ccaa4657f8246,6b409fc6-cb35-411a-bf82-08987e4ef1c2,6,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
3,02a25cddec634621822ccaa4657f8246,39598e5b-1598-4fc5-96c1-ab962fb7d99d,11,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
4,02a25cddec634621822ccaa4657f8246,82d333a5-444c-4f2c-bea4-f06dc1d2162d,16,1,non,non,582af9cc-0efa-4c6e-a407-d3d3ad15724f,child
...,...,...,...,...,...,...,...,...
12528,066a95de555443719f17ffe5732f9657,ec8a1c52-47a1-4ee6-b62f-b38050e726cf,13,2,non,non,241b91e0-3ac4-4eee-8632-ff392e586a17,child
12529,066a95de555443719f17ffe5732f9657,64c8257d-d765-4564-a8b2-cb113476bdc6,7,1,non,non,241b91e0-3ac4-4eee-8632-ff392e586a17,child
12530,066a95de555443719f17ffe5732f9657,b9893d55-a2a6-4910-a77e-3af7e8c059e3,5,2,non,non,241b91e0-3ac4-4eee-8632-ff392e586a17,child
12531,066a95de555443719f17ffe5732f9657,0830ae92-0ffb-4789-8e26-61acd6e1525e,1,2,non,non,241b91e0-3ac4-4eee-8632-ff392e586a17,child


12533

In [7]:
load_dotenv()
# get the environment variables needed
USER= config('USRCaris')
PASSWORD= config('PASSCaris')
HOSTNAME= config('HOSTCaris')
DBNAME= config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query = f'''
SELECT
	au.username,
	au.email,
	count(*) as qty
FROM
	muso_household_2022 mh
	LEFT JOIN auth_users au ON mh.created_by = au.id
	group by au.username, mh.id_patient
'''

muso_hiv = read_sql_query(query,engine,parse_dates=True)
# get the test excel file from Query

# close the pool of connection
engine.dispose()


True

In [8]:
result_hivmuso = muso_hiv.groupby(['username','email']).count()
result_hivmuso.loc['Total']= result_hivmuso.qty.sum()
result_hivmuso.reset_index(inplace=True)
result_hivmuso


,index,qty
0,"(Ernstbelabre, ernst.belabre@carisfoundationin...",221
1,"(Gustave, firma.gustave@carisfoundationintl.org)",489
2,"(June007, jeanrony.valbrun@carisfoundationintl...",588
3,"(despeignes, roselord.despeignes@carisfoundati...",169
4,"(jfodney17, fodney.joseph@carisfoundationintl....",526
5,"(josy22, josy.serafils@carisfoundationintl.org)",394
6,"(judesonvOLMAR, judeson.volmar@carisfoundation...",158
7,"(marcakernstdubois, kernst.dubois@carisfoundat...",508
8,"(suzanne, suzanne.clerge@carisfoundationintl.org)",121
9,"(viola, viola.tombeau@carisfoundationintl.org)",49
